In [1]:
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import glob
import pandas as pd

from astropy.io import fits

matplotlib.rc('xtick', labelsize=6) 
matplotlib.rc('ytick', labelsize=6) 

In [21]:
# Import the comparison data (work on this later...)
comp_name = ['cd-38_245','cs22892-52','g64-12','hd122563','hd140283','he1523-0901']
comp_color = [plt.cm.hsv(each)
              for each in np.linspace(0, 1, len(comp_name))]

comp_spec = []
for i in range(len(comp_name)):
    hdus = fits.open('/Users/xwou/Documents/Research/utilities/smhr-rpa/smh/data/spectra/'+ comp_name[i] +'.fits')
    head = hdus[0].header
    data = hdus[0].data
    hdus.close()

    df_tmp = pd.DataFrame({'wavelength': head['CRVAL1']+np.arange(len(data))*head['CDELT1'],
                           'flux': data})
    
    comp_spec.append(df_tmp.copy())
    

# Fits file only included flux data; wavelength is stored as a starting point plus delta wl along the axis

In [5]:
# Import the data
# name_spec = ['smss1_09','smss1_12','smss1_18','smss1_29','smss1_33',
#              'smss331','smss334','smss338','smss343','smss401','smss424','smss431','smss432','smss433','smss697','smss709','smss710','smss712']

# old_spec, new_spec = [], []
# for i in range(len(name_spec)):
#     old_spec.append(pd.read_csv('../data/norm_spec/'+name_spec[i]+'_old',delim_whitespace=True))
#     try:
#         new_spec.append(pd.read_csv('../data/norm_spec/'+name_spec[i]+'_rered',delim_whitespace=True))
#     except:
#         tmp_spec = pd.read_csv('../data/norm_spec/'+name_spec[i]+'_old',delim_whitespace=True)
#         tmp_spec.loc[:,'flux'] = np.zeros(len(tmp_spec))
#         new_spec.append(tmp_spec.copy())

In [40]:
# Import the data (specifying the datafile name)
name_spec = ['j1122-0209']

all_spec = []
for i in range(len(name_spec)):
    df_data = pd.read_csv('../data/MIKE_20220404_norm_spec/'+name_spec[i],delim_whitespace=True)
    all_spec.append(df_data.copy())

In [41]:
# Define the plotting function
# A4 canvas
fig_width_cm = 21                                # A4 page
fig_height_cm = 29.7
inches_per_cm = 1 / 2.54                         # Convert cm to inches
fig_width = fig_width_cm * inches_per_cm         # width in inches
fig_height = fig_height_cm * inches_per_cm       # height in inches
fig_size = [fig_width, fig_height]

# Wavelength ranges for the summary plot
start_wls = [4858.3,6559.8,5163,4305,4553,4933,4076.5,4214.5,3817,4127]
end_wls = [4864.3,6565.8,5187,4328,4555,4935,4079,4216.5,3821,4131]
elem_names = ['H-beta','H-alpha','Mg','CH','Ba','Ba','Sr','Sr','Eu','Eu']

def plot_summary(old_data,new_data,comp_data,filename,start_wls=start_wls,end_wls=end_wls,elem_names=elem_names):
    # Determine how many rows and subplots in total needed
    if new_data != None:
        assert (len(old_data) == len(new_data)) & (len(start_wls) == len(end_wls))
    N_page = len(old_data)
    N_subp = len(start_wls)
    
    # Loop through pages with each pages with a given amount of rows
    for i in range(N_page):
        f = plt.figure(figsize=fig_size)
        # For each page, plot rows and save it to a separate pdf file

        # Start plotting
        for j in range(N_subp):
            if j == 2 or j == 3:
                ax = plt.subplot(10,1,j+1)
            else:
                ax = plt.subplot(5,2,j+1)
            # Determine the starting index and ending index for a given row
            # Could just plot the whole thing all the time, but we want to save time
            start_wl, end_wl = start_wls[j], end_wls[j]

            # Plot the old spectrum
            try:
                start_ind = np.where(abs(old_data[i]['wavelength']-start_wl) <= 0.1)[0][0]
                end_ind = np.where(abs(old_data[i]['wavelength']-end_wl) <= 0.1)[0][-1]
                ax.plot(old_data[i]['wavelength'][start_ind:end_ind],old_data[i]['flux'][start_ind:end_ind],
                         ls='-',c='k',lw=0.5,label='Star',zorder=len(comp_name)+1)
                if j != 2 and j != 3:
                    ax.plot(old_data[i]['wavelength'][start_ind:end_ind],old_data[i]['flux'][start_ind:end_ind],
                            'ok', markersize= 1, fillstyle='none',zorder=len(comp_name)+1)
            except IndexError:
                print('Star',name_spec[i],'has no old spectrum at wavelength range:',str(start_wl),'to',str(end_wl))

            # Plot the new re-reduced spectrum
            if new_data != None:
                try:
                    start_ind = np.where(abs(new_data[i]['wavelength']-start_wl) <= 0.1)[0][0]
                    end_ind = np.where(abs(new_data[i]['wavelength']-end_wl) <= 0.1)[0][-1]
                    ax.plot(new_data[i]['wavelength'][start_ind:end_ind],new_data[i]['flux'][start_ind:end_ind],
                             ls='--',c='r',lw=0.5,label='New')
                except IndexError:
                    print('Star',name_spec[i],'has no rereduced spectrum at wavelength range:',str(start_wl),'to',str(end_wl))
                
            # Plot the comparison spectrum
            for k in range(len(comp_name)):
                try:
                    start_ind = np.where(abs(comp_data[k]['wavelength']-start_wl) <= 0.1)[0][0]
                    end_ind = np.where(abs(comp_data[k]['wavelength']-end_wl) <= 0.1)[0][-1]
                    ax.plot(comp_data[k]['wavelength'][start_ind:end_ind],comp_data[k]['flux'][start_ind:end_ind],
                            ls='-',c=comp_color[k],lw=0.5,label=comp_name[k],zorder=k)
                except IndexError:
                    print('Comparison star',comp_name[k],'has no old spectrum at wavelength range:',str(start_wl),'to',str(end_wl))


            ax.set_xlim([start_wl,end_wl])
            ax.set_ylim([0,1.2])
            ax.set_ylabel(elem_names[j],fontsize=12)

            # Put legend if this is the first row of the page
            if j == 1:
                ax.legend(fontsize=5)
            
        # Export the current page
#         plt.show()
        f.savefig(filename+name_spec[i]+".pdf")
        plt.close(f)

In [42]:
plot_summary(all_spec,None,comp_spec,filename='norm_spec_plot/summary_plots/summary_',
             start_wls=start_wls,end_wls=end_wls,elem_names=elem_names)